In [70]:
from pennylane import numpy as np
import pennylane as qml
import matplotlib.pyplot as plt
%matplotlib inline

np.set_printoptions(precision=5, suppress=True)

dev = qml.device("default.qubit", wires=2)

train_set = np.loadtxt(open("mock_train_set.csv"), delimiter=",")
test_set = np.loadtxt(open("mock_test_set.csv"), delimiter=",")

train_set = np.loadtxt(open("mock_train_set.csv"), delimiter=",")
test_set = np.loadtxt(open("mock_test_set.csv"), delimiter=",")

def preprocessing(data):
    data[1:,1] = np.log10(data[1:,1])
    data[1:,2] = np.log10(data[1:,2])
    data[1:,3] = np.sin(data[1:,3]*np.pi/180)
    data[1:,4] = 2 * data[1:,4] - 1
    return data

train_set = preprocessing(train_set)
test_set = preprocessing(test_set)

# Encoding using tensor products used during testing to verify our statevector is correct
def dense_angle_encoding(x):
    return np.array([
        np.cos(np.pi * x[..., 0]) * np.cos(np.pi * x[..., 2]),
        np.cos(np.pi * x[..., 0]) * np.exp(2 * np.pi * 1j * x[..., 3] ) * np.sin(np.pi * x[..., 2]),
        np.exp(2 * np.pi * 1j * x[..., 1] ) * np.sin(np.pi * x[..., 0]) * np.cos(np.pi * x[..., 2]),
        np.exp(2 * np.pi * 1j * x[..., 1] ) * np.sin(np.pi * x[..., 0]) * np.exp(2 * np.pi * 1j * x[..., 3] ) * np.sin(np.pi * x[..., 2])
    ]).T

In [71]:
# Normalizes data between -π and π
def get_norms(x):
    n_rows, n_cols = x.shape
    maxes = np.max(np.abs(x), axis=0)
    params = x / np.tile(maxes, (n_rows, 1))
    params[:,0:3] *= np.pi
    return params

print("First 3 rows of raw train_set data:\n", train_set[1:,:][0:3])

train_set = get_norms(train_set[1:])
test_set = get_norms(test_set[1:])

print("First 3 rows of normalized train_set data:\n", train_set[:][0:3])

features = train_set[:,0:4]

First 3 rows of raw train_set data:
 [[2789.26       3.         1.         0.34202   -1.     ]
 [4040.01       6.         0.         0.01745    1.     ]
 [2931.2        4.         4.         0.64279    1.     ]]
First 3 rows of normalized train_set data:
 [[ 1.75408  1.5708   0.5236   0.34202 -1.     ]
 [ 2.54064  3.14159  0.       0.01745  1.     ]
 [ 1.84335  2.0944   2.0944   0.64279  1.     ]]


In [72]:
def state_preparation(a):
    qml.RY(a[0], wires=0)
    qml.PhaseShift(a[1], wires=0)
    qml.RY(a[2], wires=1)
    qml.PhaseShift(a[3], wires=1)

In [73]:
@qml.qnode(dev)
def circuit(weights, params):
    state_preparation(params)

    for W in weights:
        layer(W)

    return qml.expval(qml.PauliZ(0))

def layer(W):
    qml.Rot(W[0, 0], W[0, 1], W[0, 2], wires=0)
    qml.Rot(W[1, 0], W[0, 1], W[1, 2], wires=1)
    qml.CNOT(wires=[1, 0])

def variational_classifier(weights, bias, params):
    return circuit(weights, params) + bias

def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss += (l - p) ** 2
    loss /= len(labels)
    return loss

def accuracy(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)
    return loss

def cost(weights, bias, features, labels):
    predictions = [variational_classifier(weights, bias, f) for f in features]
    return square_loss(labels, predictions)

In [74]:
np.random.seed(0)
num_data = len(Y)
num_train = int(0.8 * num_data)
index = np.random.permutation(range(num_data))
feats_train = features[index[:num_train]]
Y_train = Y[index[:num_train]]
feats_val = features[index[num_train:]]
Y_val = Y[index[num_train:]]

In [75]:
num_qubits = 2
num_layers = 6

weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

When training for only 60 steps, the test accuracy was somewhat low - around 84%. Training for 200 steps increased it to 93%. Next we should see if changing the circuit layout will improve accuracy.

In [76]:
from pennylane.optimize import AdamOptimizer
import torch

opt = AdamOptimizer()
batch_size = 10

# train the variational classifier
weights = weights_init
bias = bias_init
for it in range(200):
    # Update the weights by one optimizer step
    batch_index = np.random.randint(0, num_train, (batch_size,))
    feats_train_batch = feats_train[batch_index]
    Y_train_batch = Y_train[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, feats_train_batch, Y_train_batch)

    # Compute predictions on train and validation set
    predictions_train = [np.sign(variational_classifier(weights, bias, f)) for f in feats_train]
    predictions_val = [np.sign(variational_classifier(weights, bias, f)) for f in feats_val]

    # Compute accuracy on train and validation set
    acc_train = accuracy(Y_train, predictions_train)
    acc_val = accuracy(Y_val, predictions_val)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Acc train: {:0.7f} | Acc validation: {:0.7f} "
        "".format(it + 1, cost(weights, bias, features, Y), acc_train, acc_val)
    )

Iter:     1 | Cost: 1.5028249 | Acc train: 0.5125000 | Acc validation: 0.5833333 
Iter:     2 | Cost: 1.4859569 | Acc train: 0.5125000 | Acc validation: 0.6000000 
Iter:     3 | Cost: 1.4731116 | Acc train: 0.5166667 | Acc validation: 0.6000000 
Iter:     4 | Cost: 1.4620810 | Acc train: 0.5166667 | Acc validation: 0.6000000 
Iter:     5 | Cost: 1.4534776 | Acc train: 0.5166667 | Acc validation: 0.6000000 
Iter:     6 | Cost: 1.4451849 | Acc train: 0.5208333 | Acc validation: 0.6000000 
Iter:     7 | Cost: 1.4365903 | Acc train: 0.5208333 | Acc validation: 0.6000000 
Iter:     8 | Cost: 1.4271767 | Acc train: 0.5208333 | Acc validation: 0.6000000 
Iter:     9 | Cost: 1.4160046 | Acc train: 0.5208333 | Acc validation: 0.6000000 
Iter:    10 | Cost: 1.4048627 | Acc train: 0.5250000 | Acc validation: 0.6000000 
Iter:    11 | Cost: 1.3926792 | Acc train: 0.5250000 | Acc validation: 0.6000000 
Iter:    12 | Cost: 1.3821576 | Acc train: 0.5291667 | Acc validation: 0.6000000 
Iter:    13 | Co

Iter:   101 | Cost: 0.5104449 | Acc train: 0.8333333 | Acc validation: 0.8666667 
Iter:   102 | Cost: 0.5071563 | Acc train: 0.8333333 | Acc validation: 0.8666667 
Iter:   103 | Cost: 0.5043619 | Acc train: 0.8416667 | Acc validation: 0.8833333 
Iter:   104 | Cost: 0.5013679 | Acc train: 0.8416667 | Acc validation: 0.8833333 
Iter:   105 | Cost: 0.4988111 | Acc train: 0.8375000 | Acc validation: 0.8833333 
Iter:   106 | Cost: 0.4962355 | Acc train: 0.8416667 | Acc validation: 0.8833333 
Iter:   107 | Cost: 0.4941548 | Acc train: 0.8458333 | Acc validation: 0.8833333 
Iter:   108 | Cost: 0.4917389 | Acc train: 0.8625000 | Acc validation: 0.8833333 
Iter:   109 | Cost: 0.4904647 | Acc train: 0.8625000 | Acc validation: 0.8833333 
Iter:   110 | Cost: 0.4890725 | Acc train: 0.8583333 | Acc validation: 0.8833333 
Iter:   111 | Cost: 0.4876803 | Acc train: 0.8583333 | Acc validation: 0.8833333 
Iter:   112 | Cost: 0.4866103 | Acc train: 0.8583333 | Acc validation: 0.8833333 
Iter:   113 | Co

In [77]:
# process our test data like we did our train data

test_features = test_set[:, 0:4]
print("First 3 features:\n", test_features[0:3])

Y_test = test_set[:, -1]

predictions_test = [np.sign(variational_classifier(weights, bias, f)) for f in test_features]
acc_test = accuracy(Y_test, predictions_test)
print("\nTesting accuracy: {}".format(acc_test))

First 3 features:
 [[1.91088 2.0944  2.0944  0.96593]
 [2.18279 0.      1.0472  1.     ]
 [2.48824 0.      0.      0.08716]]

Testing accuracy: 0.925


In [83]:
def layer(W):
    qml.Rot(W[0, 0], W[0, 1], W[0, 2], wires=0)
    qml.Rot(W[1, 0], W[1, 2], W[1, 0], wires=0)
    qml.Rot(W[0, 1], W[0, 1], W[1, 2], wires=1)
    qml.Rot(W[1, 0], W[0, 1], W[1, 0], wires=1)
    qml.CNOT(wires=[1, 0])
    
num_qubits = 2
num_layers = 6

weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

opt = AdamOptimizer()
batch_size = 10

# train the variational classifier
weights = weights_init
bias = bias_init
for it in range(100):
    # Update the weights by one optimizer step
    batch_index = np.random.randint(0, num_train, (batch_size,))
    feats_train_batch = feats_train[batch_index]
    Y_train_batch = Y_train[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, feats_train_batch, Y_train_batch)

    # Compute predictions on train and validation set
    predictions_train = [np.sign(variational_classifier(weights, bias, f)) for f in feats_train]
    predictions_val = [np.sign(variational_classifier(weights, bias, f)) for f in feats_val]

    # Compute accuracy on train and validation set
    acc_train = accuracy(Y_train, predictions_train)
    acc_val = accuracy(Y_val, predictions_val)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Acc train: {:0.7f} | Acc validation: {:0.7f} "
        "".format(it + 1, cost(weights, bias, features, Y), acc_train, acc_val)
    )
    

predictions_test = [np.sign(variational_classifier(weights, bias, f)) for f in test_features]
acc_test = accuracy(Y_test, predictions_test)
print("\nTesting accuracy: {}".format(acc_test))

Iter:     1 | Cost: 1.4697453 | Acc train: 0.5125000 | Acc validation: 0.6000000 
Iter:     2 | Cost: 1.4224652 | Acc train: 0.5291667 | Acc validation: 0.5833333 
Iter:     3 | Cost: 1.3805219 | Acc train: 0.5291667 | Acc validation: 0.5833333 
Iter:     4 | Cost: 1.3407414 | Acc train: 0.5375000 | Acc validation: 0.5833333 
Iter:     5 | Cost: 1.3025020 | Acc train: 0.5375000 | Acc validation: 0.5833333 
Iter:     6 | Cost: 1.2644630 | Acc train: 0.5375000 | Acc validation: 0.5666667 
Iter:     7 | Cost: 1.2291224 | Acc train: 0.5416667 | Acc validation: 0.5666667 
Iter:     8 | Cost: 1.1962177 | Acc train: 0.5541667 | Acc validation: 0.5666667 
Iter:     9 | Cost: 1.1658814 | Acc train: 0.5666667 | Acc validation: 0.5666667 
Iter:    10 | Cost: 1.1340544 | Acc train: 0.5666667 | Acc validation: 0.6000000 
Iter:    11 | Cost: 1.1002966 | Acc train: 0.5666667 | Acc validation: 0.6166667 
Iter:    12 | Cost: 1.0648336 | Acc train: 0.5708333 | Acc validation: 0.6333333 
Iter:    13 | Co


Testing accuracy: 0.9666666666666667


Using more gates in our circuit caused the training to happen in fewer steps, and improved our test set accuracy to about 97%.

This model benefitted from having a deeper circuit when using the dense angle encoding.